In [3]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns

import os.path as op
import os
import glob
import shutil
from datetime import datetime
import math

from nilearn import plotting
from nilearn import image
from nilearn import masking
import nilearn


In [4]:
# get the current date/time to use in output file names 
datestring = datetime.now()
print(datestring)
timestampStr = datestring.strftime("%b%d_%Y_%H%M")
print(timestampStr)

2022-03-24 11:52:32.884688
Mar24_2022


In [5]:
# create function to write warnings/errors to file 
warningfile = 'warnings' + timestampStr + '.txt'
print(warningfile)
#writeissue(warningfile, message)
def writeissue(filename, message):
    f = open(filename, 'a')
    f.write(message + '\n\n\n') 
    f.close()

warnings.txt


In [ ]:
# Set up variables for main extraction
proj_dir = '/nese/mit/group/saxelab/projects/EMOfd/'
tier_dir = op.join(proj_dir, 'TIER')

copes_dir = op.join(tier_dir,'original_data','copes')
varcopes_dir = op.join(tier_dir,'original_data','varcopes')
zstats_dir = op.join(tier_dir,'original_data','zstats')

first_level_dir = op.join(proj_dir, 'Analysis', 'first_level_standard')

path_to_masks = op.join(tier_dir, 'analysis_data/ROI_MASKS/')
path_to_goodvoxel_masks = op.join(proj_dir, 'TIER/analysis_data/ROI_masks_goodvoxels/')



# get the info for subjects and exclusions 
master_data = op.join(proj_dir, 'data/subject_lists/EMOfd_subject_info_211026.csv')
subjects_df = pd.read_csv(master_data)


In [ ]:
# create some directories for outputs 
top_voxel_mask_outputdir = op.join(tier_dir, 'analysis_data/analyzed_ROI_data/univariate_ROI/subject_top_voxel_masks')
outputs_dir =  op.join(tier_dir, 'analysis_data/analyzed_ROI_data/univariate_ROI')
os.makedirs(outputs_dir, exist_ok = True)
os.makedirs(top_voxel_mask_outputdir, exist_ok = True)

# create output file name for CSV 
# for row-by-row (preserves per-fold)
fname_rowbyrow = op.join(outputs_dir, 'univariate_magnitude_extractions_in_ROI_' + timestampStr + '.csv')

In [ ]:
## IDENTIFY TASKS AND CONTRASTS


# set the list of tasks 
tasks = ['read', 'tomloc']


# set the contrasts for selection: 
sel_cons = [['cgf-gt-cgn', 'cgd-gt-cgn'], # for task==read: fear-selective, disgust-selective
            ['belief-gt-photo'] # for task==tomloc: ToM-selective
           ]

# ^^ note that we nest our selection contrasts 
# so that we can access all per-task selection contrasts 
# at the same index of the task in 'tasks' variable
   
    
# set the conditions for extraction: 
extr_cons = ['cgf', 'cgd', 'cgn', 'tgn']

## ^^ note: in this example, we extract the same conditions 
# regardless of which task our selection contrast comes from
# but note, these may also be nested for your analysis 

In [ ]:
# set the # of voxels to select/extract from 
# (or e.g., "whole")
# you will need to build in if-statements to determine how selection is done if not a numeric value
top_voxel_selection_methods = [100]


In [ ]:
# are you using leave-one-out "folds" 
# (this is relevant if your selection data and extracted data are non-independent)


use_folds = 1

if use_folds:
    # create output name for CSV to average across folds: 
    fname_foldsAve = op.join(outputs_dir, 'univariate_magnitude_extractions_in_ROI_' + timestampStr + '_aveAcrossFolds.csv')


In [ ]:
# GET ROI / SEARCH-SPACE MASKS 

ROIs = []
# specify which files from the original ROI masks dir to add -- this will determine which ROIs are used in subsequent analyses
ROIs += glob.glob(path_to_masks + '*.nii.gz')
print(ROIs)


ROI_names = ['rSTS', 'rTPJ', 'lSTS', 'lTPJ']
# assign names to ROIs **in the order they are listed in maskfiles array**


# note, if your ROIs are coming from an atlas, 
# you can split them beforehand using a python script loop, or using fslmaths 
# or you can load as an atlas and loop within this script
    # ^^ in that case, you will loop over the possible range of values to create masks within the main loop
        # and ROI_names should be in order of the value indices within the atlas map  


In [ ]:
# specify subject list 

# set subjects list to all subjs not listed as excluded from analyses 
filesubs = subjects_df.loc[(subjects_df.exclude_from_analysis == False), 'subjectID'].tolist()    
print(filesubs)

# alternative, if just a list of subjects from file: 
# filesubs = subjects_df.



In [7]:
## DO THE MAIN EXTRACTION

# for each type of voxel selection ... 
for selection_method in top_voxel_selection_methods:
    print('NUM TOP VOXEL METHOD: ', selection_method)
    
    # for each subject ... 
    for sub in filesubs:
        print('SUBJECT: ', sub)
    
            # for task in tasks 
                # for selection con per-task
                    # for extr con 
                    
        # for each ROI                
        for roi_file in ROIs: 
            # get the ROI name from the separate vector 
            roi = ROI_names[ROIs.index(roi_file)]
 
            print('WORKING ON ROI: ', roi)

            # load the ROI mask image  
            mask_img = image.load_img(roi_file)
            mask_img_data = mask_img.get_fdata()
            
            # find how many (non-zero) voxels are in the ROI mask
            voxels_in_roi_mask = np.sum(np.abs(mask_img_ORIG_data) > 0.0)
            
            # for each task 
            for task in tasks: 
                # get selection contrasts for this task only
                selection_cons = sel_cons[tasks.index(task)]
                
                # for each selection contrast 
                for selection_con in selection_cons:
                    
                    # IF FOLDS: USE FOLDS FORMAT TO GET CURR. SEL-COND IMAGE 
                    if use_folds:
                        selectionZ_fnames = '{}/{}_{}_fold_*_exclude_run*_con_*_{}_zstat.nii.gz'.format(zstats_dir, sub, task, selection_con)
                    else: # OTHERWISE USE THE EXCLUDE-NONE FOLD 
                        # note: this still assumes 2nd level was run with folds; if not, the filename won't contain "fold" or "exclude_none"; check yours
                        selectionZ_fnames = '{}/{}_{}_fold_*_exclude_none_con_*_{}_zstat.nii.gz'.format(zstats_dir, sub, task, selection_con)
                    
                    
                    matches_selectionZ_fnames = glob.glob(selectionZ_fnames)
                    num_folds = len(matches_selectionZ_fnames)

                    if num_folds == 0:
                        continue
                        # skip subjects who have NO files;
                        
                    else: 
                        #get z stat for fROI definition contrast (F>B) per fold
                        
                        for fold in range(0,num_folds):
                            
                            # load the z-stat image (for voxel selection)
                            matches_combZ_current_fold = matches_selectionZ_fnames[fold]
                            combZ_img = image.load_img(matches_combZ_current_fold)

                            # mask our train contrast image using the ROI mask 
                            masked_combZ_img = image.math_img("img1 * img2", img1 = combZ_img, img2 = mask_img)

                            # actually get the data (real values) from the masked copes 
                            masked_combZ_data = masked_combZ_img.get_fdata()
                            
                            # SET ALL ZEROS TO NAN BEFORE GRABBING  TOP 100; ENSURES 0S->NANS NOT INCLUDED IN TOP 100, MEANING TOP 100 CAN INCLUDE NEG. VALUES
                            masked_combZ_data[masked_combZ_data == 0.00000000] = np.nan

                            #NEXT: DROP THE ZEROS
                            nanned_combZ_data = masked_combZ_data.copy()
                            nanned_combZ_data[nanned_combZ_data == 0] = np.nan
                            combZ_roi_nans_inds = (-nanned_combZ_data).argsort(axis = None)


                            ## GET WHICH VOXELS ARE TOP 
                            if selection_method == 'whole':
                                nVox = np.count_nonzero(voxels_in_roi_mask)
                                masked_combZ_data[np.unravel_index(combZ_roi_nans_inds[nVox:], masked_combZ_data.shape)] = np.nan
                            elif selection_method == 100:
                                combZ_roi_nans_inds = (-np.absolute(nanned_combZ_data)).argsort(axis = None)
                                nVox = min(selection_method, np.count_nonzero(voxel_filter))
                                masked_combZ_data[np.unravel_index(combZ_roi_nans_inds[nVox:], masked_combZ_data.shape)] = np.nan


                            # GET BINARY MASKS (1,0) OF WHICH VOXELS ARE TOP IN OUR TRAIN DATA 
                            top_voxel_mask = masked_combZ_data.copy()
                            top_voxel_mask[~np.isnan(top_voxel_mask)] = 1
                            top_voxel_mask[np.isnan(top_voxel_mask)] = 0


                            # save the top voxel masks to files 
                            top_voxel_fname = '{}/{}_task-{}_fold-{}_{}_top-{}_contrast-{}_from-zstat.nii.gz'.format(top_voxel_mask_outputdir, sub, task, fold, roi, selection_method, selection_con)
                            top_voxel_img = image.new_img_like(combZ_img, top_voxel_mask)
                            top_voxel_img.to_filename(top_voxel_fname)


                            ### NOW MOVING ON TO PULLING BETAS FROM TOP VOXELS 


                            for contrast in extr_cons: 
                                
                                if use_folds: 
                                    # get the excluded run left out from the current fold
                                    splits_1 = test.split('exclude_')
                                    splits_2 = splits_1[1].split('_')
                                    excluded_run = splits_2[0]
                                    extr_cope_fname = '{}/{}/{}/model/{}/{}_cope.nii.gz'.format(first_level_dir, sub, task, excluded_run, contrast)
                                    
                                else: 
                                    # get the full data for this contrast from TIER copes dir: 
                                    # again, this still assumes you ran second level w/ folds, but we'll use the exclude-none output here: 
                                    extr_cope_fname = '{}/{}_{}_fold_*_exclude_none_con_*_{}_zstat.nii.gz'.format(copes_dir, sub, task, contrast)
                                
                                matches_extract_cope_fname = glob.glob(extr_cope_fname)

                                # load the image 
                                extract_cope_img = image.load_img(matches_extract_cope_fname[0])

                                # mask w/ TOP VOXEL MASK
                                top_voxel_masked_extract_fold_cope_img = image.math_img("img1 * img2", img1 = extract_cope_img, img2 = top_voxel_img)
                                # get data 
                                top_voxel_masked_extract_cope_data = top_voxel_masked_extract_fold_cope_img.get_fdata()

                
                                # set 0s to NaN again
                                top_voxel_masked_extract_cope_data[top_voxel_masked_extract_cope_data == 0] = np.nan
                                # take the mean of the top voxels 
                                mean_top_vox = np.nanmean(top_voxel_masked_extract_cope_data)

                                
                                if use_folds:
                                    df_mag_currentrow = pd.DataFrame({'subjectID': sub, 
                                                                      'task' : task,
                                                                      'roi' : roi,
                                                                      'voxels_in_roi_mask': voxels_in_roi_mask,
                                                                      'top_voxel_selection_method' : selection_method,
                                                                      'selection_contrast' : selection_con, 
                                                                      'extraction_contrast' : contrast, 
                                                                      'fold_number': fold,
                                                                      'excluded_run_number': excluded_run,
                                                                      'mean_topvoxels_extracted_cope' : mean_top_vox}, index=[0])
                                else: 
                                    df_mag_currentrow = pd.DataFrame({'subjectID': sub,
                                                                  'task' : task,
                                                                  'roi' : roi,
                                                                  'voxels_in_roi_mask': voxels_in_roi_mask,
                                                                  'top_voxel_selection_method' : selection_method,
                                                                  'selection_contrast' : selection_con, 
                                                                  'extraction_contrast' : contrast, 
                                                                  'mean_topvoxels_extracted_cope' : mean_top_vox}, index=[0])

                                if not os.path.isfile(fname_rowbyrow):
                                    # if file doesn't exist, write it with column headers 
                                    df_mag_currentrow.to_csv(fname_rowbyrow, index=False, header='column_names')
                                else: 
                                    # else append w/o column headers 
                                    df_mag_currentrow.to_csv(fname_rowbyrow, mode='a', index=False, header=False)


['sub-SAXEEMOfd04', 'sub-SAXEEMOfd05', 'sub-SAXEEMOfd06', 'sub-SAXEEMOfd07', 'sub-SAXEEMOfd08', 'sub-SAXEEMOfd09', 'sub-SAXEEMOfd10', 'sub-SAXEEMOfd13', 'sub-SAXEEMOfd14', 'sub-SAXEEMOfd15', 'sub-SAXEEMOfd16', 'sub-SAXEEMOfd17', 'sub-SAXEEMOfd18', 'sub-SAXEEMOfd19', 'sub-SAXEEMOfd20', 'sub-SAXEEMOfd22', 'sub-SAXEEMOfd23', 'sub-SAXEEMOfd24', 'sub-SAXEEMOfd25', 'sub-SAXEEMOfd27', 'sub-SAXEEMOfd29', 'sub-SAXEEMOfd30', 'sub-SAXEEMOfd33', 'sub-SAXEEMOfd34', 'sub-SAXEEMOfd35', 'sub-SAXEEMOfd36', 'sub-SAXEEMOfd37', 'sub-SAXEEMOfd38', 'sub-SAXEEMOfd39', 'sub-SAXEEMOfd40', 'sub-SAXEEMOfd41', 'sub-SAXEEMOfd42', 'sub-SAXEEMOfd43', 'sub-SAXEEMOfd45', 'sub-SAXEEMOfd46', 'sub-SAXEEMOfd47', 'sub-SAXEEMOfd48', 'sub-SAXEEMOfd49', 'sub-SAXEEMOfd50', 'sub-SAXEEMOfd51', 'sub-SAXEEMOfd52', 'sub-SAXEEMOfd53']
NUM TOP VOXEL TYPE:  100
SUBJECT:  sub-SAXEEMOfd04
WORKING ON ROI:  disgust_190611_out
(91, 109, 91)
GOOD VOXELS IN ROI PROPORTION:
1.0
WORKING ON FOLD:  1
WORKING ON CONTRAST:  con_1_tgn-gt-cgn
(91, 1

KeyboardInterrupt: 

In [7]:
dfmag_whole = pd.read_csv(fname_rowbyrow)


if use_folds:
    df_acrossRuns = dfmag_whole.groupby(['participantID', 'roi', 'contrast','method'])['mean_extracted_cope'].mean().to_frame(name = 'mean_topvoxels_leftout_copes_averaged').reset_index()
    df_acrossRuns = df_acrossRuns.drop(columns=["method"])
    df_acrossRuns = df_acrossRuns.sort_values(by=['participantID', 'roi', 'contrast'])
    df_acrossRuns.to_csv(fname_foldsAve, index=False, header='column_names')


In [8]:
print("DONE!")

DONE!
